<a href="https://colab.research.google.com/github/AnastasiaBrinati/Progetto-ML-23-24/blob/main/task2_federato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 2 FEDERATED

###Imports

In [1]:
!pip install -q flwr["simulation"] tensorflow
!pip install -q flwr_datasets["vision"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.

Let's also install Matplotlib so we can make some plots once the simulation is completed

In [2]:
!pip install matplotlib

Next, we import the required dependencies. The most important imports are Flower (`flwr`) and TensorFlow:

In [3]:
from typing import Dict, List, Tuple

import tensorflow as tf

import flwr as fl
from flwr.common import Metrics
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth

from datasets import Dataset
from flwr_datasets import FederatedDataset

VERBOSE = 0
NUM_CLIENTS = 100

In [4]:
import tensorflow as tf

from tensorflow import keras
from keras import layers

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

## Model

It is good practice to normalize features that use different scales and ranges.

One reason this is important is because the features are multiplied by the model weights. So, the scale of the outputs and the scale of the gradients are affected by the scale of the inputs.

Although a model *might* converge without feature normalization, normalization makes training much more stable.

Note: There is no advantage to normalizing the one-hot features—it is done here for simplicity. For more details on how to use the preprocessing layers, refer to the [Working with preprocessing layers](https://www.tensorflow.org/guide/keras/preprocessing_layers) guide and the [Classify structured data using Keras preprocessing layers](../structured_data/preprocessing_layers.ipynb) tutorial.

### Building the DNN

In [6]:
def get_model():

    """Constructs a model suitable."""
    # Define input layers for all the features
    mwg_input = tf.keras.layers.Input(shape=(1,), name='MWG')
    nwg_input = tf.keras.layers.Input(shape=(1,), name='NWG')
    kwg_input = tf.keras.layers.Input(shape=(1,), name='KWG')
    mdimc_input = tf.keras.layers.Input(shape=(1,), name='MDIMC')
    ndimc_input = tf.keras.layers.Input(shape=(1,), name='NDIMC')
    mdima_input = tf.keras.layers.Input(shape=(1,), name='MDIMA')
    ndimb_input = tf.keras.layers.Input(shape=(1,), name='NDIMB')
    kwi_input = tf.keras.layers.Input(shape=(1,), name='KWI')
    vwm_input = tf.keras.layers.Input(shape=(1,), name='VWM')
    vwn_input = tf.keras.layers.Input(shape=(1,), name='VWN')
    strm_input = tf.keras.layers.Input(shape=(1,), name='STRM')
    strn_input = tf.keras.layers.Input(shape=(1,), name='STRN')
    sa_input = tf.keras.layers.Input(shape=(1,), name='SA')
    sb_input = tf.keras.layers.Input(shape=(1,), name='SB')


    # Concatenate the input tensors
    concatenated_inputs = tf.keras.layers.concatenate([mwg_input, nwg_input,kwg_input,mdimc_input,
                                                       ndimc_input,mdima_input,ndimb_input,kwi_input,
                                                       vwm_input,vwn_input,strm_input,strn_input,
                                                       sa_input,sb_input
                                                       ])

    # Define the rest of the model
    x = tf.keras.layers.Flatten()(concatenated_inputs)

    #      'num_layers': 2,
    #      'units_0': 128,
    #      'activation_0': 'sigmoid',
    #      'units_1': 128,
    #      'activation_1': 'sigmoid',
    #      'units_2': 112,
    #      'activation_2': 'PReLU',
    #      'learning_rate': 0.01,
    #      'units_3': 96,
    #      'activation_3': 'sigmoid'

    x = tf.keras.layers.Dense(128, activation="sigmoid")(x)
    x = tf.keras.layers.Dense(128, activation="sigmoid")(x)

    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    # Construct the model
    model = tf.keras.models.Model(inputs=[mwg_input, nwg_input,kwg_input,mdimc_input,
                                          ndimc_input,mdima_input,ndimb_input,kwi_input,
                                          vwm_input,vwn_input,strm_input,strn_input,
                                          sa_input,sb_input], outputs=output)

    # Compile the model
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.01), metrics=["mse", "mape", "mean_squared_logarithmic_error","R2Score"])

    return model

# come era prima:
      #model = keras.Sequential([
      #layers.Flatten(),
      #layers.Dense(16, activation='relu', name="uno"),  #input_shape=(2, 32,15)
      #layers.Dense(1, name="ultimo"),

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#dnn_model = get_model(normalizer)
dnn_model = get_model()
dnn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 MWG (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 NWG (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 KWG (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 MDIMC (InputLayer)          [(None, 1)]                  0         []                            
                                                                                              

## Clients

With that out of the way, let's move on to the interesting bits. Federated learning systems consist of a server and multiple clients. In Flower, we create clients by implementing subclasses of `flwr.client.Client` or `flwr.client.NumPyClient`. We use `NumPyClient` in this tutorial because it is easier to implement and requires us to write less boilerplate.

To implement the Flower client, we create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`:

- `get_parameters`: Return the current local model parameters
- `fit`: Receive model parameters from the server, train the model parameters on the local data, and return the (updated) model parameters to the server
- `evaluate`: Received model parameters from the server, evaluate the model parameters on the local data, and return the evaluation result to the server

We mentioned that our clients will use TensorFlow/Keras for the model training and evaluation. Keras models provide methods that make the implementation straightforward: we can update the local model with server-provides parameters through `model.set_weights`, we can train/evaluate the model through `fit/evaluate`, and we can get the updated model parameters through `model.get_weights`.

Let's see a simple implementation:

In [8]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainset, valset) -> None:
        # Create model
        self.model = get_model()
        self.trainset = trainset
        self.valset = valset

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.trainset, epochs=1, verbose=VERBOSE)
        return self.model.get_weights(), len(self.trainset), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        results = self.model.evaluate(self.valset) #, verbose=VERBOSE)

        loss = results[0]
        mse = results[1]
        mape = results[2]
        mean_squared_logarithmic_error = results[3]
        r2score = results[4]

        # return {"loss": loss, "mse": mse, "mape": mape, "mean_squared_logarithmic_error":mean_squared_logarithmic_error, "R2Score": r2score}
        return loss, len(self.valset), {"loss": loss, "mse": mse, "mape": mape, "mean_squared_logarithmic_error": mean_squared_logarithmic_error, "R2Score": r2score}

Our class `FlowerClient` defines how local training/evaluation will be performed and allows Flower to call the local training/evaluation through `fit` and `evaluate`. Each instance of `FlowerClient` represents a *single client* in our federated learning system. Federated learning systems have multiple clients (otherwise, there's not much to federate, is there?), so each client will be represented by its own instance of `FlowerClient`. If we have, for example, three clients in our workload, we'd have three instances of `FlowerClient`. Flower calls `FlowerClient.fit` on the respective instance when the server selects a particular client for training (and `FlowerClient.evaluate` for evaluation).

In this notebook, we want to simulate a federated learning system with 100 clients on a single machine. This means that the server and all 100 clients will live on a single machine and share resources such as CPU, GPU, and memory. Having 100 clients would mean having 100 instances of `FlowerClient` in memory. Doing this on a single machine can quickly exhaust the available memory resources, even if only a subset of these clients participates in a single round of federated learning.

In addition to the regular capabilities where server and clients run on multiple machines, Flower, therefore, provides special simulation capabilities that create `FlowerClient` instances only when they are actually necessary for training or evaluation. To enable the Flower framework to create clients when necessary, we need to implement a function called `client_fn` that creates a `FlowerClient` instance on demand. Flower calls `client_fn` whenever it needs an instance of one particular client to call `fit` or `evaluate` (those instances are usually discarded after use). Clients are identified by a client ID, or short `cid`. The `cid` can be used, for example, to load different local data partitions for each client

##Strategy

In [9]:
"""Returns a function to construct a client.
    The VirtualClientEngine will execute this function whenever a client is sampled by the strategy to participate. """

def get_client_fn(dataset: FederatedDataset):

    def client_fn(cid: str) -> fl.client.Client:
        """Construct a FlowerClient with its own dataset partition."""

        # Extract partition for client with id = cid
        # !!!!!!!!
        client_dataset = dataset.load_partition(int(cid), "train")

        # Now let's split it into train (80%) and validation (20%)
        client_dataset_splits = client_dataset.train_test_split(test_size=0.2)

        train_set = client_dataset_splits["train"].to_tf_dataset(columns=['MWG', 'NWG', 'KWG', 'MDIMC', 'NDIMC', 'MDIMA', 'NDIMB', 'KWI', 'VWM',
       'VWN', 'STRM', 'STRN', 'SA', 'SB'], label_cols=['avg_runs'], batch_size=32)
        val_set = client_dataset_splits["test"].to_tf_dataset(columns=['MWG', 'NWG', 'KWG', 'MDIMC', 'NDIMC', 'MDIMA', 'NDIMB', 'KWI', 'VWM',
       'VWN', 'STRM', 'STRN', 'SA', 'SB'], label_cols=['avg_runs'], batch_size=64)


        # Create and return client
        return FlowerClient(train_set, val_set).to_client()

    return client_fn

In [10]:
"""Aggregation function for (federated) evaluation metrics, i.e. those returned by the client's evaluate() method."""

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply metric of each client by number of examples used

    #losses = [num_examples * m["loss"] for num_examples, m in metrics]
    mses = [num_examples * m["mse"] for num_examples, m in metrics]
    mapes = [num_examples * m["mape"] for num_examples, m in metrics]
    mean_squared_logarithmic_errors = [num_examples * m["mean_squared_logarithmic_error"] for num_examples, m in metrics]
    R2Scores = [num_examples * m["R2Score"] for num_examples, m in metrics]

    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"mse": sum(mses) / sum(examples), "mape": sum(mapes) / sum(examples), "mean_squared_logarithmic_error": sum(mean_squared_logarithmic_errors) / sum(examples),"R2Score": sum(R2Scores) / sum(examples)}

In [11]:
"""Returns an evaluation function for server-side (i.e. centralised) evaluation."""

def get_evaluate_fn(testset: Dataset):
    # The `evaluate` function will be called after every round by the strategy
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):
        model = get_model()  # Construct the model
        model.set_weights(parameters)  # Update model with the latest parameters
        aiuto = model.evaluate(testset) #, verbose=VERBOSE)
        # return {"loss": res['loss'], "mse": res['mse'], "mape": res['mape'], "mean_squared_logarithmic_error": res['mean_squared_logarithmic_error'], "R2Score": res['R2Score']}
        # return {"loss": loss, "mse": res[0], "mape": res[1], "mean_squared_logarithmic_error": res[2], "R2Score": res[3]}
        # return {"loss": aiuto[0], "mse": aiuto[1], "mape": aiuto[2], "mean_squared_logarithmic_error": aiuto[3], "R2Score": aiuto[4]}
        return aiuto[0], {"mse": aiuto[1], "mape": aiuto[2], "mean_squared_logarithmic_error": aiuto[3], "R2Score": aiuto[4]}

    return evaluate

We now have `FlowerClient` which defines client-side training and evaluation, and `client_fn`, which allows Flower to create `FlowerClient` instances whenever it needs to call `fit` or `evaluate` on one particular client. The last step is to start the actual simulation using `flwr.simulation.start_simulation`.

In [33]:

# Simulation parameters
settings = {
    "VERBOSE": 0,
    "NUM_CLIENTS": 100,
    "FRACTION_FIT": 0.8,
    "FRACTION_EVALUATE": 0.2,
    "MIN_FIT_CLIENTS": 5,
    "MIN_EVALUATE_CLIENTS": 2,
    "MIN_AVAILABLE_CLIENTS_FRACTION": 0.5,
    "STRATEGY_TYPE": 'QFedAvg', # QFedAvg, FedAvg, FedProx
    "NUM_ROUNDS": 10
}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Create FedAvg strategy

def get_strategy(centralized_testset):

  strategy1 = fl.server.strategy.FedAvg(
    fraction_fit=0.1,  # Sample 10% of available clients for training
    fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=int(NUM_CLIENTS * 0.5),  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # aggregates federated metrics
    evaluate_fn=get_evaluate_fn(centralized_testset),  # global evaluation function
  )

  strategy2 = fl.server.strategy.FedProx(
    proximal_mu=1.0,
    fraction_fit=settings["FRACTION_FIT"],
    fraction_evaluate=settings["FRACTION_EVALUATE"],
    min_fit_clients=settings["MIN_FIT_CLIENTS"],
    min_evaluate_clients=settings["MIN_EVALUATE_CLIENTS"],
    min_available_clients=int(settings["NUM_CLIENTS"] * settings["MIN_AVAILABLE_CLIENTS_FRACTION"]),
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=get_evaluate_fn(centralized_testset),
  )

  strategy3 = fl.server.strategy.QFedAvg(
    fraction_fit=settings["FRACTION_FIT"],
    fraction_evaluate=settings["FRACTION_EVALUATE"],
    min_fit_clients=settings["MIN_FIT_CLIENTS"],
    min_evaluate_clients=settings["MIN_EVALUATE_CLIENTS"],
    min_available_clients=int(settings["NUM_CLIENTS"] * settings["MIN_AVAILABLE_CLIENTS_FRACTION"]),
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=get_evaluate_fn(centralized_testset),
  )

  return strategy3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Start the Simulation

The function `start_simulation` accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy. The strategy encapsulates the federated learning approach/algorithm, for example, *Federated Averaging* (FedAvg).

Flower comes with a number of built-in strategies, but we can also use our own strategy implementations to customize nearly all aspects of the federated learning approach. For this example, we use the built-in `FedAvg` implementation and customize it using a few basic parameters. The last step is the actual call to `start_simulation` which - you guessed it - actually starts the simulation.

We can use [Flower Datasets](https://flower.dev/docs/datasets/) to effortlessly obtain an off-the-shelf partitioned dataset or partition one that isn't pre-partitioned. Let's choose MNIST.

In [43]:
# Enable GPU growth in your main process
enable_tf_gpu_growth()

# Download our dataset and partition it
gpus_fds = FederatedDataset(dataset="anastasiafrosted/gpus", partitioners={"train": NUM_CLIENTS})
# Get the whole test set for centralised evaluation
#print(column_names)
centralized_testset = gpus_fds.load_full("test").to_tf_dataset(columns=['MWG', 'NWG', 'KWG', 'MDIMC', 'NDIMC', 'MDIMA', 'NDIMB', 'KWI', 'VWM',
       'VWN', 'STRM', 'STRN', 'SA', 'SB'], label_cols=['avg_runs'], batch_size=64)

# get the strategy
strategy = get_strategy(centralized_testset)

/usr/local/lib/python3.10/dist-packages/flwr_datasets/utils.py:84: UserWarning: The currently tested dataset are ['mnist', 'cifar10', 'fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet']. Given: anastasiafrosted/gpus.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:401: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [44]:
# With a dictionary, you tell Flower's VirtualClientEngine that each
# client needs exclusive access to these many resources in order to run
client_resources = {"num_cpus": 2, "num_gpus": 0.0}

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=get_client_fn(gpus_fds),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
    actor_kwargs={
        "on_actor_init_fn": enable_tf_gpu_growth  # Enable GPU growth upon actor init.
    },
)

INFO flwr 2024-03-10 16:13:10,259 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-03-10 16:13:16,466	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-10 16:13:19,371 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7848021198.0, 'object_store_memory': 3924010598.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7848021198.0, 'object_store_memory': 3924010598.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-03-10 16:13:19,378 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-10 1

755/755 [==============================] - 17s 21ms/step - loss: 1.0214 - mse: 4.0234 - mape: 1154.4869 - mean_squared_logarithmic_error: 0.3425 - r2_score: -0.0287


INFO flwr 2024-03-10 16:13:44,472 | server.py:94 | initial parameters (loss, other metrics): 1.0213792324066162, {'mse': 4.023429870605469, 'mape': 1154.4869384765625, 'mean_squared_logarithmic_error': 0.3425126075744629, 'R2Score': -0.028672218322753906}
INFO:flwr:initial parameters (loss, other metrics): 1.0213792324066162, {'mse': 4.023429870605469, 'mape': 1154.4869384765625, 'mean_squared_logarithmic_error': 0.3425126075744629, 'R2Score': -0.028672218322753906}
INFO flwr 2024-03-10 16:13:44,484 | server.py:104 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2024-03-10 16:13:44,500 | server.py:222 | fit_round 1: strategy sampled 80 clients (out of 100)
DEBUG:flwr:fit_round 1: strategy sampled 80 clients (out of 100)
DEBUG flwr 2024-03-10 16:16:54,154 | server.py:236 | fit_round 1 received 80 results and 0 failures
DEBUG:flwr:fit_round 1 received 80 results and 0 failures


755/755 [==============================] - 11s 14ms/step - loss: 1.0214 - mse: 4.0234 - mape: 1154.4869 - mean_squared_logarithmic_error: 0.3425 - r2_score: -0.0287


WARNING flwr 2024-03-10 16:17:05,723 | qfedavg.py:225 | No fit_metrics_aggregation_fn provided


...

755/755 [==============================] - 13s 16ms/step - loss: 0.9593 - mse: 4.2511 - mape: 562.3711 - mean_squared_logarithmic_error: 0.3796 - r2_score: -0.0869


INFO flwr 2024-03-10 16:50:59,860 | server.py:125 | fit progress: (10, 0.9592827558517456, {'mse': 4.251059532165527, 'mape': 562.37109375, 'mean_squared_logarithmic_error': 0.3795928657054901, 'R2Score': -0.08687043190002441}, 2235.3615421699997)
INFO:flwr:fit progress: (10, 0.9592827558517456, {'mse': 4.251059532165527, 'mape': 562.37109375, 'mean_squared_logarithmic_error': 0.3795928657054901, 'R2Score': -0.08687043190002441}, 2235.3615421699997)
DEBUG flwr 2024-03-10 16:50:59,865 | server.py:173 | evaluate_round 10: strategy sampled 20 clients (out of 100)
DEBUG:flwr:evaluate_round 10: strategy sampled 20 clients (out of 100)


7/7 [==============================] - 1s 14ms/step - loss: 0.8523 - mse: 3.1417 - mape: 368.3676 - mean_squared_logarithmic_error: 0.3230 - r2_score: -0.0770


DEBUG flwr 2024-03-10 16:51:23,138 | server.py:187 | evaluate_round 10 received 20 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 20 results and 0 failures
INFO flwr 2024-03-10 16:51:23,144 | server.py:153 | FL finished in 2258.645742778999
INFO:flwr:FL finished in 2258.645742778999
INFO flwr 2024-03-10 16:51:23,149 | app.py:226 | app_fit: losses_distributed [(1, 1.0129086941480636), (2, 1.0069327652454376), (3, 0.9882948637008667), (4, 1.0111054211854935), (5, 0.9915450513362885), (6, 0.9768500298261642), (7, 1.025166916847229), (8, 0.9411063879728317), (9, 0.9830107033252716), (10, 0.9518591582775116)]
INFO:flwr:app_fit: losses_distributed [(1, 1.0129086941480636), (2, 1.0069327652454376), (3, 0.9882948637008667), (4, 1.0111054211854935), (5, 0.9915450513362885), (6, 0.9768500298261642), (7, 1.025166916847229), (8, 0.9411063879728317), (9, 0.9830107033252716), (10, 0.9518591582775116)]
INFO flwr 2024-03-10 16:51:23,152 | app.py:227 | app_fit: metrics_distributed_fit {}


7/7 [==============================] - 1s 14ms/step - loss: 1.1426 - mse: 6.1315 - mape: 349.4578 - mean_squared_logarithmic_error: 0.4663 - r2_score: -0.1100


# PLOT

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
  fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(19, 5))

  ax1.plot(pd.DataFrame(history.c_loss), label='loss_centralizzata')
  ax1.plot(pd.DataFrame(history.loss_avg), label='loss_avg')
  ax1.plot(pd.DataFrame(history.loss_prox), label='loss_prox')
  ax1.plot(pd.DataFrame(history.loss_qavg), label='loss_qavg')

  ax2.plot(pd.DataFrame(history.c_mse), label='mse_centralized')
  ax2.plot(pd.DataFrame(history.mse_avg), label='mse_avg')
  ax2.plot(pd.DataFrame(history.mse_prox), label='mse_prox')
  ax2.plot(pd.DataFrame(history.mse_qavg), label='mse_qavg')

  ax3.plot(pd.DataFrame(history.c_mape), label='mape_centralized')
  ax3.plot(pd.DataFrame(history.mape_avg), label='mape_avg')
  ax3.plot(pd.DataFrame(history.mape_prox), label='mape_prox')
  ax3.plot(pd.DataFrame(history.mape_qavg), label='mape_qavg')

  ax4.plot(pd.DataFrame(history.c_msle), label='msle_centralized')
  ax4.plot(pd.DataFrame(history.msle_avg), label='msle_avg')
  ax4.plot(pd.DataFrame(history.msle_prox), label='msle_prox')
  ax4.plot(pd.DataFrame(history.msle_qavg), label='msle_qavg')

  ax5.plot(pd.DataFrame(history.c_r2), label='r2_centralized')
  ax5.plot(pd.DataFrame(history.r2_avg), label='r2_avg')
  ax5.plot(pd.DataFrame(history.r2_prox), label='r2_prox')
  ax5.plot(pd.DataFrame(history.r2_qavg), label='r2_qavg')

  ax1.set(xlabel='Epoch', ylabel='loss')
  ax1.legend(loc="upper right")
  ax1.grid(True)

  ax2.set(xlabel='Epoch', ylabel='mse')
  ax2.legend(loc="upper right")
  ax2.grid(True)

  ax3.set(xlabel='Epoch', ylabel='mape')
  ax3.legend(loc="upper right")
  ax3.grid(True)

  ax4.set(xlabel='Epoch', ylabel='mean_squared_logarithmic_error')
  ax4.legend(loc="upper right")
  ax4.grid(True)

  ax5.set(xlabel='Epoch', ylabel='r2_score')
  ax5.legend(loc="upper right")
  ax5.grid(True)

  fig.tight_layout()

#### CENTRALE

In [ ]:
# initialize list of lists
data_centrale = [[0, 0, 0, 0, 0, 0],
                [1,0.654691,3.148931,84.439003,0.158293,0.179876],
                [2,0.638235,3.136050,79.750023,0.154645,0.183230],
                [3,0.631250,3.132224,77.709427,0.152647,0.184227],
                [4,0.620261,3.123100,80.765099,0.148184,0.186603],
                [5,0.616670,3.121987,78.399124,0.147527,0.186893],
                [6,0.620576,3.122755,93.155891,0.147896,0.186693],
                [7,0.613600,3.121242,78.342842,0.147210,0.187087],
                [8,0.613441,3.121131,79.306496,0.147167,0.187116],
                [9,0.611118,3.120584,76.801743,0.146897,0.187258],
                [10,0.611208,3.120517,77.030296,0.146838,0.187276]]

# Create the pandas DataFrame
h_centrale = pd.DataFrame(data_centrale, columns=['epoch','c_loss', 'c_mse', 'c_mape', 'c_msle','c_r2']).drop(0)

h_centrale

,epoch,c_loss,c_mse,c_mape,c_msle,c_r2
1,1,0.654691,3.148931,84.439003,0.158293,0.179876
2,2,0.638235,3.136050,79.750023,0.154645,0.183230
3,3,0.631250,3.132224,77.709427,0.152647,0.184227
4,4,0.620261,3.123100,80.765099,0.148184,0.186603
5,5,0.616670,3.121987,78.399124,0.147527,0.186893
6,6,0.620576,3.122755,93.155891,0.147896,0.186693
7,7,0.613600,3.121242,78.342842,0.147210,0.187087
8,8,0.613441,3.121131,79.306496,0.147167,0.187116
9,9,0.611118,3.120584,76.801743,0.146897,0.187258
10,10,0.611208,3.120517,77.030296,0.146838,0.187276


# SIMULAZIONE 5 CLIENTS

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})
h_avg = h_fedavg.drop(0)
h_avg

In [ ]:
data_5_average = [[0, 0, 0, 0, 0],
              [-0.159772,	4.536199,	100.001656,	0.487906,	0.934440],
              [-0.159774,	4.536206,	99.999771,	0.487910,	0.934440],
              [-0.159774,	4.536205,	99.999786,	0.487910,	0.934440],
              [-0.159772,	4.536200,	100.000877,	0.487907,	0.934440],
              [-0.159772,	4.536198,	100.001709,	0.487906,	0.934440],
              [-0.159754,	4.536129,	100.024345,	0.487872,	0.934437],
              [0.095176,	3.539023,	254.785141,	0.231574,	0.789448],
              [0.168450,	3.252428,	122.435867,	0.172460,	0.700098],
              [0.173803,	3.231492,	139.457962,	0.166702,	0.685768],
              [0.173894,	3.231137,	101.962616,	0.167250,	0.680446]]

# Create the pandas DataFrame
h5_average = pd.DataFrame(data_5_average, columns=['r2_avg', 'mse_avg', 'mape_avg', 'msle_avg','loss_avg']).drop(0)

h5_average

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})

h_prox = h_prox.drop(0)
h_prox

In [ ]:
data_5_proxima = [[0, 0, 0, 0, 0],
    [-0.159773,	4.536204,	99.999832,	0.487908,	0.934440],
    [-0.159771,	4.536196,	100.002647,	0.487904,	0.934440],
    [-0.158326,	4.530542,	100.980331,	0.485681,	0.933808],
    [0.164455,	3.268056,	149.804108,	0.175392,	0.708424],
    [0.173493,	3.232704,	119.574417,	0.166720,	0.683495],
    [0.174212,	3.229892,	110.613647,	0.166439,	0.680148],
    [0.174251,	3.229741,	102.588463,	0.166694,	0.678648],
    [0.174063,	3.230473,	98.588257,	0.167074,	0.678383],
    [0.174425,	3.229059,	98.732056,	0.166520,	0.677230],
    [0.174464,	3.228906,	100.152863,	0.166480,	0.677253]]

# Create the pandas DataFrame
h5_proxima = pd.DataFrame(data_5_proxima, columns=['r2_prox', 'mse_prox', 'mape_prox', 'msle_prox','loss_prox']).drop(0)

h5_proxima

#### QAVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})

h_qavg = h_fedQavg.drop(0)
h_qavg

In [ ]:
data_5_qaverage = [ [0, 0, 0, 0, 0],
    [-0.038929,	4.063546,	1040.778809,	0.345072,	1.006306],
    [-0.044094,	4.083748,	978.655701,	0.346872,	0.998679],
    [-0.049749,	4.105868,	915.088745,	0.349496,	0.991300],
    [-0.055025,	4.126504,	858.891418,	0.352478,	0.985156],
    [-0.060640,	4.148464,	802.461853,	0.356186,	0.979595],
    [-0.066540,	4.171540,	746.391968,	0.360625,	0.974560],
    [-0.072246,	4.193858,	694.489624,	0.365406,	0.970081],
    [-0.077753,	4.215397,	646.320923,	0.370449,	0.966047],
    [-0.082950,	4.235724,	602.545837,	0.375580,	0.962494],
    [-0.088348,	4.256840,	558.664307,	0.381279,	0.959051]]


 # Create the pandas DataFrame
h5_qaverage = pd.DataFrame(data_5_qaverage, columns=['r2_qavg', 'mse_qavg', 'mape_qavg', 'msle_qavg','loss_qavg']).drop(0)
h5_qaverage

# SIMULAZIONE 10 CLIENTS

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})
h_avg = h_fedavg.drop(0)
h_avg

In [ ]:
data_10_average = [ [0, 0, 0, 0, 0],
  [-0.159768,	4.536181,	100.008499,	0.487897,	0.934440],
  [-0.159773,	4.536201,	100.000008,	0.487907,	0.934440],
  [-0.159770,	4.536192,	100.004105,	0.487902,	0.934440],
  [-0.159465,	4.534999,	100.231705,	0.487408,	0.934316],
  [0.145597,	3.341815,	242.549881,	0.192786,	0.743602],
  [0.172998,	3.234639,	152.504593,	0.167351,	0.690544],
  [0.173969,	3.230843,	105.542000,	0.166795,	0.680298],
  [0.173977,	3.230810,	108.258484,	0.167205,	0.681332],
  [0.173879,	3.231196,	91.323975,	0.167707,	0.678554],
  [0.173640,	3.232130,	104.274834,	0.167719,	0.683049]]


 # Create the pandas DataFrame
h10_average = pd.DataFrame(data_10_average, columns=['r2_qavg', 'mse_qavg', 'mape_qavg', 'msle_qavg','loss_qavg']).drop(0)
h10_average

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})
h_prox = h_prox.drop(0)
h_prox

In [ ]:
data_10_proxima = [[0, 0, 0, 0, 0],
    [-0.159773,	4.536201,	100.000259,	0.487907,	0.934440],
    [-0.159773,	4.536201,	100.000587,	0.487907,	0.934440],
    [-0.159766,	4.536175,	100.009506,	0.487894,	0.934439],
    [-0.068974,	4.181060,	118.042313,	0.386204,	0.888390],
    [0.170903,	3.242836,	145.703079,	0.169003,	0.694669],
    [0.173511,	3.232634,	111.831459,	0.167015,	0.682743],
    [0.174152,	3.230126,	106.305733,	0.166629,	0.679604],
    [0.174308,	3.229515,	106.238319,	0.166608,	0.679228],
    [0.174417,	3.229089,	101.954605,	0.166529,	0.677882],
    [0.174371,	3.229271,	99.536942,	0.166657,	0.677419]]


# Create the pandas DataFrame
h10_proxima = pd.DataFrame(data_10_proxima, columns=['r2_prox', 'mse_prox', 'mape_prox', 'msle_prox','loss_prox']).drop(0)

h10_proxima

#### Q AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})
h_qavg = h_fedQavg.drop(0)
h_qavg

In [ ]:
data_10_qaverage = [ [0, 0, 0, 0, 0],
  [-0.005093,	3.931203,	1658.599976,	0.359964,	1.101884],
  [-0.007134,	3.939188,	1588.970215,	0.355774,	1.089777],
  [-0.009641,	3.948995,	1517.632446,	0.352034,	1.077690],
  [-0.012628,	3.960676,	1445.581665,	0.348873,	1.065847],
  [-0.016163,	3.974501,	1371.580688,	0.346307,	1.054089],
  [-0.020172,	3.990185,	1297.480347,	0.344467,	1.042741],
  [-0.024852,	4.008488,	1220.603516,	0.343398,	1.031461],
  [-0.029889,	4.028190,	1145.836914,	0.343219,	1.020971],
  [-0.035364,	4.049602,	1071.925171,	0.343951,	1.011085],
  [-0.041095,	4.072019,	1000.736694,	0.345560,	1.002041]]

 # Create the pandas DataFrame
h10_qaverage = pd.DataFrame(data_10_qaverage, columns=['r2_qavg', 'mse_qavg', 'mape_qavg', 'msle_qavg','loss_qavg']).drop(0)
h10_qaverage

# SIMULAZIONE 100 CLIENTS

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})

h_avg = h_fedavg.drop(0)
h_avg

In [ ]:
data_average = [[0, 0, 0, 0, 0],
    [-0.159755,	4.536134,	100.026421,	0.487875,	0.934440],
    [-0.159772,	4.536198,	100.001831,	0.487905,	0.934440],
    [-0.159762,	4.536160,	100.014900,	0.487887,	0.934439],
    [-0.145003,	4.478434,	107.778694,	0.466935,	0.927268],
    [0.165347,	3.264564,	139.972153,	0.174531,	0.706536],
    [0.173709,	3.231861,	124.542587,	0.166392,	0.684164],
    [0.173527,	3.232573,	137.992813,	0.167373,	0.687349],
    [0.174131,	3.230210,	106.839127,	0.166867,	0.680561],
    [0.173989,	3.230766,	93.856712,	0.167519,	0.678331],
    [0.173803,	3.231493,	117.283958,	0.167247,	0.683554]]

# Create the pandas DataFrame
h_average = pd.DataFrame(data_average, columns=['r2_avg', 'mse_avg', 'mape_avg', 'msle_avg','loss_avg']).drop(0)

h_average

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})

h_prox = h_prox.drop(0)
h_prox

In [ ]:
data_proxima = [[0, 0, 0, 0, 0],
    [-0.159760,	4.536153,	100.019691,	0.487883,	0.934439],
    [-0.159761,	4.536155,	100.016273,	0.487885,	0.934438],
    [-0.066254,	4.170422,	123.066887,	0.384461,	0.885895],
    [0.169462,	3.248470,	131.076019,	0.170648,	0.697338],
    [0.173644,	3.232113,	113.603127,	0.166872,	0.682718],
    [0.174282,	3.229618,	108.092812,	0.166477,	0.679581],
    [0.174300,	3.229549,	100.192329,	0.166766,	0.678160],
    [0.174375,	3.229254,	96.198891,	0.166825,	0.677046],
    [0.174396,	3.229172,	99.913826,	0.166705,	0.677745],
    [0.174443,	3.228990,	100.616287,	0.166603,	0.677719]]

# Create the pandas DataFrame
h_proxima = pd.DataFrame(data_proxima, columns=['r2_prox', 'mse_prox', 'mape_prox', 'msle_prox','loss_prox']).drop(0)
h_proxima

#### QAVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['R2Score']).drop([0], axis=1).rename(columns={1: 'r2_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mse']).drop([0], axis=1)], axis=1).rename(columns={1: 'mse_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mape']).drop([0], axis=1)], axis=1).rename(columns={1: 'mape_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['mean_squared_logarithmic_error']).drop([0], axis=1)], axis=1).rename(columns={1: 'msle_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})

h_qavg = h_fedQavg.drop(0)
h_qavg

In [ ]:
data_qaverage = [ [0, 0, 0, 0, 0],
                [-0.034129,	4.044774,	1080.976440,	0.343173,	1.011565],
                [-0.039911,	4.067389,	1009.415894,	0.344734,	1.002501],
                [-0.045783,	4.090356,	942.036987,	0.347087,	0.994473],
                [-0.051844,	4.114060,	877.082764,	0.350228,	0.987184],
                [-0.058080,	4.138453,	814.192810,	0.354128,	0.980678],
                [-0.063909,	4.161250,	758.587280,	0.358335,	0.975573],
                [-0.069688,	4.183853,	706.047241,	0.363008,	0.970987],
                [-0.075778,	4.207674,	653.270447,	0.368445,	0.966562],
                [-0.081590,	4.230405,	604.900879,	0.374091,	0.962633],
                [-0.086870,	4.251060,	562.371094,	0.379593,	0.959283]]

 # Create the pandas DataFrame
h_qaverage = pd.DataFrame(data_qaverage, columns=['r2_qavg', 'mse_qavg', 'mape_qavg', 'msle_qavg','loss_qavg']).drop(0)

h_qaverage